# Credit Card Fraud Detection

- Task: Identify fraudulent credit card transactions.
- transactions performed in Sept 2013 (2 days) by european card holders
- highly unbalanced data: 492 frauds out of 284,807 [0.172%]
- Binary Classification problem


In [2]:
#Import DL libraries:
import keras
keras.__version__

'2.4.3'

In [3]:
#Import other libraries:
import numpy as np
import pandas as pd

## Step 1: Load/analyze Data: 

In [4]:
credit_data = pd.read_csv('creditcard.csv')
credit_data.head(n=10)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
5,2.0,-0.425966,0.960523,1.141109,-0.168252,0.420987,-0.029728,0.476201,0.260314,-0.568671,...,-0.208254,-0.559825,-0.026398,-0.371427,-0.232794,0.105915,0.253844,0.081080,3.67,0
6,4.0,1.229658,0.141004,0.045371,1.202613,0.191881,0.272708,-0.005159,0.081213,0.464960,...,-0.167716,-0.270710,-0.154104,-0.780055,0.750137,-0.257237,0.034507,0.005168,4.99,0
7,7.0,-0.644269,1.417964,1.074380,-0.492199,0.948934,0.428118,1.120631,-3.807864,0.615375,...,1.943465,-1.015455,0.057504,-0.649709,-0.415267,-0.051634,-1.206921,-1.085339,40.80,0
8,7.0,-0.894286,0.286157,-0.113192,-0.271526,2.669599,3.721818,0.370145,0.851084,-0.392048,...,-0.073425,-0.268092,-0.204233,1.011592,0.373205,-0.384157,0.011747,0.142404,93.20,0
9,9.0,-0.338262,1.119593,1.044367,-0.222187,0.499361,-0.246761,0.651583,0.069539,-0.736727,...,-0.246914,-0.633753,-0.120794,-0.385050,-0.069733,0.094199,0.246219,0.083076,3.68,0


In [5]:
credit_data.shape

(284807, 31)

In [6]:
#Columns:
#- class -> Output var, value: 0 or 1 (fraud)
#- time -> seconds elasped b/w current and first transaction
#- amount -> transaction amount/cost sensitive learning
# V1-V8 -> principal components obtained from pca.
#all columns of type float64 except class
credit_data.dtypes

Time      float64
V1        float64
V2        float64
V3        float64
V4        float64
V5        float64
V6        float64
V7        float64
V8        float64
V9        float64
V10       float64
V11       float64
V12       float64
V13       float64
V14       float64
V15       float64
V16       float64
V17       float64
V18       float64
V19       float64
V20       float64
V21       float64
V22       float64
V23       float64
V24       float64
V25       float64
V26       float64
V27       float64
V28       float64
Amount    float64
Class       int64
dtype: object

## Step 2: Fill missing values:

In [7]:
credit_data.isnull().sum()
#No null/missing value found

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

In [8]:
#Deleting the duplicates
credit_data.duplicated().sum()

1081

In [9]:
credit_data.drop_duplicates(inplace=True)

In [10]:
credit_data.shape

(283726, 31)

## Step 3: Standarize input variables:

In [11]:

from sklearn.preprocessing import StandardScaler
scale = StandardScaler()
credit_data[['Time', 'Amount']] = scale.fit_transform(credit_data[['Time', 'Amount']])


In [12]:

X = credit_data.iloc[:, :-1].values   #other columns
y = credit_data.iloc[:, -1].values     #class


In [13]:
#Step 4: Split into 50% Training, 30% Test and 20%:

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size= 0.3, random_state= 0)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.287, random_state=0)


In [14]:
print(x_train.shape)
print(x_test.shape)
print(x_val.shape)

(141607, 30)
(85118, 30)
(57001, 30)


In [15]:
#(57001/283726)*100

In [16]:
print(y_train.shape)
print(y_test.shape)
print(y_val.shape)

(141607,)
(85118,)
(57001,)


In [17]:
#Step 5: Build input model:
#-input Layer (No. of features )
#-3 hidden layers including 10,8,6 unit 
#-Output Layer with activation function relu/tanh (check by experiment)

from keras import models, layers

model = models.Sequential()
model.add(layers.Dense(20, activation='relu', input_dim=30))
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(8, activation='relu'))
model.add(layers.Dense(6, activation='relu'))
model.add(layers.Dense(2, activation='tanh'))


In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 20)                620       
_________________________________________________________________
dense_1 (Dense)              (None, 10)                210       
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 88        
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 54        
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 14        
Total params: 986
Trainable params: 986
Non-trainable params: 0
_________________________________________________________________


In [19]:
#Step 6: Compilation Step (select loss & metrics):

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [20]:
#Step 7: Train the Model with Epochs (100):

model.fit(x_train, y_train, batch_size=20, epochs=100, validation_data=(x_val, y_val))


Epoch 1/100
7081/7081 [==============================] - 63s 9ms/step - loss: 0.0163 - accuracy: 0.4581 - val_loss: 0.0174 - val_accuracy: 0.3908
Epoch 2/100
7081/7081 [==============================] - 54s 8ms/step - loss: 0.0161 - accuracy: 0.6561 - val_loss: 0.0176 - val_accuracy: 0.8029
Epoch 3/100
7081/7081 [==============================] - 53s 7ms/step - loss: 0.0173 - accuracy: 0.9119 - val_loss: 0.0178 - val_accuracy: 0.9590
Epoch 4/100
7081/7081 [==============================] - 53s 8ms/step - loss: 0.0169 - accuracy: 0.9448 - val_loss: 0.0182 - val_accuracy: 0.9366
Epoch 5/100
7081/7081 [==============================] - 49s 7ms/step - loss: 0.0166 - accuracy: 0.8888 - val_loss: 0.0173 - val_accuracy: 0.7868
Epoch 6/100
7081/7081 [==============================] - 60s 8ms/step - loss: 0.0159 - accuracy: 0.9292 - val_loss: 0.0175 - val_accuracy: 0.9881
Epoch 7/100
7081/7081 [==============================] - 44s 6ms/step - loss: 0.0159 - accuracy: 0.9789 - val_loss: 0.0173 -

In [21]:
#Step 8: If the model gets overfit tune your model
#by changing the units , No. of layers , epochs , add dropout layer or add Regularizer according to the need .


In [22]:
#Step 9: Evaluation Step: Prediction should be > 92% 
myscore = model.evaluate(x_test, y_test, batch_size=20)
print('The accuracy score is ', myscore[1] * 100, '%')

852/852 [==============================] - 1s 1ms/step - loss: 0.0156 - accuracy: 0.9994
The accuracy score is  99.93655681610107 %


In [ ]:
#accuracy in training after 100 epochs: 99.94
#accuracy in testing = 99.93
#So, there is no need to tune model